# Federated Learning with Flower project

In [3]:
import torch as th
import os
import numpy as np
import torchvision

In [4]:
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
from torch import optim
from collections import OrderedDict

%matplotlib inline

In [5]:
import syft as sy

W0708 21:06:01.534593 32884 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.


In [6]:
hook= sy.TorchHook(th)

In [7]:
#create workers
w1 = sy.VirtualWorker(hook, id="w1")
w2 = sy.VirtualWorker(hook, id="w2")
secure_worker3 = sy.VirtualWorker(hook, id="secure_worker3")

In [8]:
#share pointers between workers
w1.add_workers([w2, secure_worker3])
w2.add_workers([w1, secure_worker3])
secure_worker3.add_workers([w1, w2])

W0708 21:06:02.137126 32884 base.py:624] Worker w2 already exists. Replacing old worker which could cause                     unexpected behavior
W0708 21:06:02.140121 32884 base.py:624] Worker secure_worker3 already exists. Replacing old worker which could cause                     unexpected behavior
W0708 21:06:02.142116 32884 base.py:624] Worker w1 already exists. Replacing old worker which could cause                     unexpected behavior
W0708 21:06:02.143116 32884 base.py:624] Worker secure_worker3 already exists. Replacing old worker which could cause                     unexpected behavior
W0708 21:06:02.145109 32884 base.py:624] Worker w1 already exists. Replacing old worker which could cause                     unexpected behavior
W0708 21:06:02.146106 32884 base.py:624] Worker w2 already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker3 #tensors:0>

In [19]:
# Dataset
#!dir flower_data\train

In [8]:
data_dir = 'flower_data'
# number of subprocesses to use for data loading
#num_workers = 1
# how many samples per batch to load
batch_size = 68
# percentage of training set to use as validation
valid_size = 0.2

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '\\train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '\\valid', transform=test_transforms)

trainloader = th.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = th.utils.data.DataLoader(test_data, batch_size=batch_size)

In [11]:
w1_train_data = train_data[0:3276]
w1_test_data = train_data[0:409]

ValueError: too many values to unpack (expected 2)